test1　　演算(+,*)のテスト

In [1]:
#　colaboraory用: Google drive をマウントする
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# colaboratory用: フォルダを移る
%cd "drive/My Drive/Colab Notebooks"

/content/drive/My Drive/Colab Notebooks


フォルダは自分の指定のものに変更して下さい。

In [3]:
%cd "ctorch210929/MiniCTorch_Prototype/notebook"

/content/drive/My Drive/Colab Notebooks/ctorch210929/MiniCTorch_Prototype/notebook


In [4]:
! pip install lark-parser

     |████████████████████████████████| 103 kB 6.7 MB/s 


In [5]:
import sys
sys.path.append("../")

import json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import minictorch.generator as GN
import minictorch.converter as CV

In [6]:
class Net(nn.Module):
   def __init__(self):
      super(Net, self).__init__()
      self.t = torch.tensor(np.array([[1.0, 2], [3, 4]]))
           
   def forward(self, x):
      f1 = 10 * x * x * self.t
      f2 = 5 * x
      y = f1 + f2
      return y

In [7]:
def generate_json( json_path, input_x ):

   model = Net()

   model.eval()
   with torch.no_grad():
      GN.generate_minictorch_file( model, input_x, json_path )

   return model

In [ ]:
"""
# 210904 move to converter.py
def convert_json( project, folder, model, input_x, json_path, rand_flag=0 ):

    cpp_fname   = project + ".cpp"
    param_fname = project + "_param.cpp"
    cpp_path    = folder + "/" + cpp_fname
    param_path  = folder + "/" + param_fname
    make_path   = folder + "/" + "Makefile"
  
    # save json file
    print( "[JSON]", json_path )
    fp = open( json_path )
    obj = json.load(fp)

    # save parameter file
    code1 = CV.c_param_generator( obj, model, input_x )
    if len(code1) > 0:
       print( "[PARAM]", param_path )
       ofparam = open( param_path, "w" )
       ofparam.write( code1 )
    else:
       param_fname=""

    #save cpp file
    code2 = CV.c_code_generator( obj, model, rand_flag )

    print("[CPP] ", cpp_path )
    ofp = open( cpp_path, "w" )
    ofp.write( code2 )

    # save make file
    make_code = CV.makefile_generator( project )

    print( "[MAKE]", make_path )
    makefp = open( make_path, "w" )
    makefp.write( make_code )
"""

In [9]:
x = torch.tensor(np.array([[1.0, 2], [3, 4]]), requires_grad=True)
print(x)

project = "test1"

json_path = './network/' + project +'.json'

model = generate_json( json_path, x )


tensor([[1., 2.],
        [3., 4.]], dtype=torch.float64, requires_grad=True)


In [11]:
CV.convert_json( project, "./src", model, x, json_path )

[JSON] ./network/test1.json
[PARAM] ./src/test1_param.cpp
{'name': 'input/x', 'op': 'IO Node', 'in': [], 'output_id': 0, 'shape': [2, 2], 'out': [2, 3, 7], 'sorted_id': 0}
{'name': 'Net/4', 'op': 'prim::Constant', 'in': [], 'output_id': 0, 'shape': [], 'constant_value': 10.0, 'out': [2], 'sorted_id': 1}
{'name': 'Net/5', 'op': 'aten::mul', 'in': [0, 1], 'output_id': 0, 'shape': [2, 2], 'out': [3], 'sorted_id': 2}
{'name': 'Net/6', 'op': 'aten::mul', 'in': [2, 0], 'output_id': 0, 'shape': [2, 2], 'out': [5], 'sorted_id': 3}
{'name': 'Net/7', 'op': 'prim::Constant', 'in': [], 'output_id': 0, 'shape': [2, 2], 'constant_value': [1.0, 2.0, 3.0, 4.0], 'out': [5], 'sorted_id': 4}
{'name': 'Net/f1', 'op': 'aten::mul', 'in': [3, 4], 'output_id': 0, 'shape': [2, 2], 'out': [9], 'sorted_id': 5}
{'name': 'Net/9', 'op': 'prim::Constant', 'in': [], 'output_id': 0, 'shape': [], 'constant_value': 5.0, 'out': [7], 'sorted_id': 6}
{'name': 'Net/f2', 'op': 'aten::mul', 'in': [0, 6], 'output_id': 0, 'shap

In [12]:
# check code
model.train()
y = model( x )
print(y)
y.sum().backward()
print(x.grad)

tensor([[ 15.,  90.],
        [285., 660.]], dtype=torch.float64, grad_fn=<AddBackward0>)
tensor([[ 25.,  85.],
        [185., 325.]], dtype=torch.float64)


In [15]:
!g++ -std=c++14 ./src/test1.cpp ./src/test1_param.cpp -D_NOTEBOOK -I ../../../xtensor -lcblas -o ./src/test1

(注意) xtensorにxtensor関連のincludeを置いています。各自の環境に合わせて変更して下さい。

In [16]:
! ./src/test1

### forward computation ...
{{  15.,   90.},
 { 285.,  660.}}
### backward computation ...
input_grad{{  25.,   85.},
 { 185.,  325.}}
